In [11]:
import os
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

In [12]:
class_names = ['Safe', 'Medium', 'Dangerous', 'Super Dangerous']

In [13]:
df = pd.read_csv('data_mapping.csv')

df2 = df.iloc[:-1, :]
ds_test, ds_train =  np.array_split(df2, 2)

print("Test dataset sample: ")
print(ds_test.head(5))

print("Train dataset sample: ")
print(ds_train.head(5))

ds_train_batch = ds_train.head(32)

Test dataset sample: 
   Unnamed: 0 county_code  month  day  hour  \
0           0      Newton      1    1     0   
1           1      Newton      1    1     1   
2           2      Newton      1    1     2   
3           3      Newton      1    1     3   
4           4      Newton      1    1     4   

                                              crimes  
0  [4, 1, 4, 1, 1, 1, 1, 1, 1, 3, 3, 1, 3, 3, 1, ...  
1  [4, 1, 2, 4, 1, 1, 2, 2, 2, 1, 2, 4, 4, 4, 2, ...  
2  [4, 3, 3, 4, 2, 3, 4, 4, 4, 1, 4, 4, 2, 2, 2, ...  
3  [2, 4, 4, 4, 2, 1, 2, 4, 4, 4, 2, 3, 2, 4, 3, ...  
4  [4, 3, 1, 4, 4, 1, 2, 4, 4, 4, 1, 4, 4, 4, 4, ...  
Train dataset sample: 
       Unnamed: 0 county_code  month  day  hour  \
21168       21168     Olympic      7    1     0   
21169       21169     Olympic      7    1     1   
21170       21170     Olympic      7    1     2   
21171       21171     Olympic      7    1     3   
21172       21172     Olympic      7    1     4   

                                   

In [14]:
# features, labels = next(iter(ds_train_batch))
features, labels = ds_train_batch.iloc[:, 0:5], ds_train_batch.iloc[:, 5]
print(features.head(5))
print(labels.head(5))

       Unnamed: 0 county_code  month  day  hour
21168       21168     Olympic      7    1     0
21169       21169     Olympic      7    1     1
21170       21170     Olympic      7    1     2
21171       21171     Olympic      7    1     3
21172       21172     Olympic      7    1     4
21168    [3, 2, 2, 2, 4, 4, 4, 4, 4, 2, 2, 4, 4, 4, 4, ...
21169    [1, 4, 3, 4, 4, 1, 4, 1, 3, 4, 4, 2, 4, 3, 2, ...
21170    [2, 4, 4, 2, 4, 4, 4, 1, 4, 1, 4, 4, 3, 4, 4, ...
21171    [2, 4, 4, 4, 2, 4, 2, 4, 4, 2, 4, 4, 2, 2, 2, ...
21172    [3, 4, 2, 1, 2, 1, 4, 2, 1, 4, 4, 4, 4, 1, 4, ...
Name: crimes, dtype: object


In [18]:
print(labels[21172][4])

labels2 = {}
for index in labels.index:
    labels2 = [int(numeric_string) for numeric_string in labels[index]]

y = list()
for index, i in labels2.iteritems():
    y.append((sum(i))/(len(i))) # finding the average severity
plt.scatter(features.loc[:,"hour"],
            y,
            c=labels,
            cmap='viridis')

plt.xlabel("Body Mass")
plt.ylabel("Culmen Length")
plt.show()

4


ValueError: invalid literal for int() with base 10: '['

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(4,)),  # input shape required
  tf.keras.layers.Dense(10, activation=tf.nn.relu),
  tf.keras.layers.Dense(3)
])

In [ ]:
predictions = model(features)
predictions[:5]

tf.nn.softmax(predictions[:5])

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[0.2565303 , 0.30331326, 0.4401565 ],
       [0.2471776 , 0.3263538 , 0.42646858],
       [0.22262622, 0.28859237, 0.4887814 ],
       [0.27415708, 0.3131517 , 0.41269124],
       [0.28524905, 0.33240163, 0.38234925]], dtype=float32)>

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
def loss(model, x, y, training):
  # training=training is needed only if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  y_ = model(x, training=training)

  return loss_object(y_true=y, y_pred=y_)

l = loss(model, features, labels, training=False)
print("Loss test: {}".format(l))

Loss test: 1.072019338607788


In [ ]:
# change the weights based on the loss. 
def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets, training=True)
  return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

In [ ]:
loss_value, grads = grad(model, features, labels)

print("Step: {}, Initial Loss: {}".format(optimizer.iterations.numpy(),
                                          loss_value.numpy()))

optimizer.apply_gradients(zip(grads, model.trainable_variables))

print("Step: {},         Loss: {}".format(optimizer.iterations.numpy(),
                                          loss(model, features, labels, training=True).numpy()))

Step: 0, Initial Loss: 1.072019338607788
Step: 1,         Loss: 1.0699684619903564


In [ ]:
## Note: Rerunning this cell uses the same model parameters

# Keep results for plotting
train_loss_results = []
train_accuracy_results = []

num_epochs = 201

for epoch in range(num_epochs):
  epoch_loss_avg = tf.keras.metrics.Mean()
  epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

  # Training loop - using batches of 32
  for x, y in ds_train_batch:
    # Optimize the model
    loss_value, grads = grad(model, x, y)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Track progress
    epoch_loss_avg.update_state(loss_value)  # Add current batch loss
    # Compare predicted label to actual label
    # training=True is needed only if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    epoch_accuracy.update_state(y, model(x, training=True))

  # End epoch
  train_loss_results.append(epoch_loss_avg.result())
  train_accuracy_results.append(epoch_accuracy.result())

  if epoch % 50 == 0:
    print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))

Epoch 000: Loss: 1.062, Accuracy: 35.581%
Epoch 050: Loss: 0.623, Accuracy: 79.775%
Epoch 100: Loss: 0.437, Accuracy: 86.517%
Epoch 150: Loss: 0.319, Accuracy: 96.255%
Epoch 200: Loss: 0.231, Accuracy: 97.004%
